# Quantized Deployment and Throughput/Accuracy Testing
This notebook deploys the quantized model on an FPGA and performs throughput and accuracy testing.
If the desired deployment is on a PYNQ board, this notebook should be run from the PYNQ host.

In [1]:
from output_final.driver.finn.core.datatype import DataType
from driver_base import FINNExampleOverlay
import numpy as np
import proc_for_infer as pfi
from utils_pynq import load_split_np_data

In [9]:
#set up model info
# dictionary describing the I/O of the FINN-generated accelerator
io_shape_dict = {
    # FINN DataType for input and output tensors
    "idt" : DataType.UINT4,
    "odt" : DataType.UINT4,
    # shapes for input and output tensors (NHWC layout)
    "ishape_normal" : (1, 14),
    "oshape_normal" : (1, 1),
    # folded / packed shapes below depend on idt/odt and input/output
    # PE/SIMD parallelization settings -- these are calculated by the
    # FINN compiler.
    "ishape_folded" : (1, 1, 14),
    "oshape_folded" : (1, 1, 1),
    "ishape_packed" : (1, 1, 7),
    "oshape_packed" : (1, 1, 1)
}
platform = "zynq-iodma"
bitfile = "output_final/deploy/bitfile/finn-accel.bit"
input_file = "data/X_train.npy"
output_comparison = "data/Y_train.npy"
runtime_weights = "output_final/deploy/driver/runtime_weights/"
batch_size = 1

In [10]:
#create accelerator
accel = FINNExampleOverlay(bitfile_name=bitfile, platform=platform, 
                          io_shape_dict=io_shape_dict, batch_size=batch_size, fclk_mhz=.1,
                          runtime_weight_dir=runtime_weights)

/usr/local/lib/python3.6/dist-packages/pynq/ps.py:464: UserWarning: Setting frequency to the closest possible value 0.37793MHz.
  round(freq_high / q0, 5)))


### Throughput Test
Run a basic throughput test and print the results

In [11]:
res = accel.throughput_test()
print(res)

{'runtime[ms]': 2.942800521850586, 'throughput[images/s]': 339.8123632828324, 'DRAM_in_bandwidth[Mb/s]': 0.0023786865429798265, 'DRAM_out_bandwidth[Mb/s]': 0.00033981236328283236, 'fclk[mhz]': 0.377925, 'batch_size': 1, 'fold_input[ms]': 5.125999450683594e-05, 'pack_input[ms]': 0.006728172302246094, 'copy_input_data_to_device[ms]': 0.00025534629821777344, 'copy_output_data_from_device[ms]': 8.749961853027344e-05, 'unpack_output[ms]': 0.0010151863098144531, 'unfold_output[ms]': 3.0040740966796875e-05}


### Accuracy Test
Perform inference on testing dataset and compare to actual values.

In [6]:
#accel.batch_size(229538)
print(accel.ishape_normal)

(1, 14)


In [7]:
datasets = load_split_np_data()
print(datasets)

X_train shape: (229538, 14)
X_val shape: (12752, 14)
X_test shape: (12752, 14)
Y_train shape: (229538, 1)
Y_val shape: (12752, 1)
Y_test shape: (12752, 1)
Using saved split data
[array([[  5.80000000e+01,   2.00000000e+00,   3.69370341e+00, ...,
          1.64524792e+03,   1.56832642e+03,   9.73000366e+02],
       [  7.30000000e+01,   3.00000000e+00,   1.63631487e+00, ...,
          1.09457141e+03,   8.94265137e+02,   7.37251587e+02],
       [  7.40000000e+01,   2.00000000e+00,   1.15914593e+01, ...,
          4.53916113e+03,   1.46776880e+03,   7.66278015e+02],
       ..., 
       [  6.40000000e+01,   2.00000000e+00,   9.21862221e+00, ...,
          1.73535828e+03,   1.19223145e+03,   6.71937500e+02],
       [  7.10000000e+01,   2.00000000e+00,   7.65338302e-01, ...,
          8.91962585e+02,   1.62368567e+03,   8.88209412e+02],
       [  9.10000000e+01,   2.00000000e+00,   8.17640972e+00, ...,
          2.95410669e+03,   1.74744006e+03,   8.36169189e+02]]), array([[  5.70000000e+01, 

In [ ]:
inps = datasets[1]
exp_out = datasets[4]
valid_size = len(exp_out)
num_batches = int(valid_size/batch_size)
running_error_square = 0
for i in range(0, num_batches):
    batch = inps[(i*batch_size):((i+1)*batch_size)]
    batch_exp_out = exp_out[(i*batch_size):((i+1)*batch_size)]
    #print(batch.shape)
    proc_batch = pfi.preproc(batch)
    #proc_batch = np.flip(proc_batch, axis=1)
    batch_out = accel.execute(proc_batch)
    batch_proc_out = pfi.postproc(batch_out)
    batch_errs = batch_proc_out-batch_exp_out
    print("exp: " + str(batch_exp_out) + " act: " + str(batch_proc_out) + " preproc: " + str(batch_out))
    batch_sq_errs = batch_errs*batch_errs
    running_error_square += sum(batch_sq_errs)
    #batch = valid[(i*batch_size):((i+1)*batch_size)]
    #inp = batch[0]
    #inp = FACILE_preproc(inp.float())
    #inp = inp.numpy()
    #inp_dict = {in_tensor : inp}
    #exp_out = batch[1].numpy()
    #out_dict = exe_onnx(model, inp_dict)
    #out = out_dict[out_tensor]
    #out = tensor(out,dtype=float32)
    #out = FACILE_postproc(out)
    #print(out)
print("MSE: " + str(running_error_square / (num_batches * batch_size)))

exp: [[ 132.65643311]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 21.67700386]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 95.71929932]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 35.5912323]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 42.45626068]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 25.850317]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 126.25019836]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 27.163414]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 79.58286285]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.

exp: [[ 74.70278168]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 135.40950012]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 32.41735077]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 39.0557785]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 24.95316696]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 27.62182808]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 29.22259521]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 70.69464111]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 45.79159164]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 54.07184601]] act: [[ 135.3068]]

exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 65.65914154]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 35.13806534]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 112.31454468]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 19.46315193]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 17.66941261]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 50.04506683]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 27.3862381]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 94.57099915]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 36.98365784]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 21.82725143]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 135.3068]] 

exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 36.2709198]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 44.38499832]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 19.62145615]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 106.57025909]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 22.47159958]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 44.75529861]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 31.04143715]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 25.21374512]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]


exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 39.53383255]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 30.53726578]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 29.73623276]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 40.41942215]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 16.92591286]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 11.26241493]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 53.5637

...

exp: [[ 156.35987854]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 28.92668343]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 62.40148926]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 32.79830933]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 100.625]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 18.71617699]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 39.83082962]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 32.96302032]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 53.27587509]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 119.64745331]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 101.4801]] prep

exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 12.12572002]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 25.30288124]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 74.58386993]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 29.08814049]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 78.58259583]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 30.24786758]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 24.41954422]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 26.95284843]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 44.01160812]] act: [[ 135.3068]] preproc: [[

exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 16.69643211]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 25.98331451]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 25.01633072]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 122.5193634]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 19.20035172]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.51498634]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 53.03713608]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 30.43156815]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 26.36484528]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 74.6583786]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 67.6534]] prepr

exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 34.4257164]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 23.0334816]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 29.81223869]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 68.36445618]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 54.15450287]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 193.09182739]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 34.36061859

exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 203.90942383]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 53.51796722]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 23.24823189]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 31.33195114]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 21.92536163]] act: [[ 135.